In [1]:
import fwk.neural_nets as nn
import fwk.optimizers as optim
import fwk.data as data
import torch
from collections import OrderedDict
import matplotlib.pyplot as plt


# Disable auto_grad
torch.set_grad_enabled(False)

# Global variables
N_EPOCHS = 1000

In [2]:
X_train, Y_train, Y_train_hot = data.generate_data()
X_test, Y_test, Y_test_hot = data.generate_data()

In [60]:
class CustomNet(nn.Module):

    def __init__(self):
        self.f = nn.Sequential(OrderedDict(
            {
                'linear 1': nn.Linear(2, 25),
                'relu 1': nn.Tanh(),
                'linear 2': nn.Linear(25, 25),
                'relu 2': nn.Tanh(),
                'linear 3': nn.Linear(25, 25),
                'relu 3': nn.Tanh(),
                'linear 4': nn.Linear(25, 2),
                'last act': nn.Tanh(),
            }
        )
        )

    def backward(self, d_loss):
        return self.f.backward(d_loss)

    def forward(self, x):
        return self.f.forward(x)
    
    def zero_grad(self):
        return self.f.zero_grad()
    
    def param(self):
        return self.f.param()

In [78]:
def train(X_train, Y_train_hot, model, optimizer, criterion, n_epochs=100, batch_size=10, verbose=False):
    for epoch in range(n_epochs):
        loss = 0
        
        for batch in range(0, len(X_train), batch_size):

            # Build batch 
            X_batch = X_train.narrow(0, batch, batch_size)
            Y_batch = Y_train_hot.narrow(0, batch, batch_size)
            
            # Forward pass
            output = model.forward(X_batch) # Model prediction
            loss += criterion.forward(output, Y_batch)
            
            # Backward pass
            model.zero_grad()
            grad = criterion.backward()
            model.backward(grad) # Compute the gradients with predictions
            optimizer.step() # Optimize parameters
        if verbose:
            print("Epoch {}: Total loss={}".format(epoch, loss))
        

In [79]:
model = CustomNet()
criterion = nn.MSE()
optimizer = optim.SGD(model.param())

In [80]:
train(X_train, Y_train_hot, model, optimizer, criterion, verbose=True)

Epoch 0: Total loss=592.1658325195312
Epoch 1: Total loss=331.9757995605469
Epoch 2: Total loss=277.70635986328125
Epoch 3: Total loss=250.56549072265625
Epoch 4: Total loss=213.40252685546875
Epoch 5: Total loss=189.43829345703125
Epoch 6: Total loss=177.2915496826172
Epoch 7: Total loss=157.93325805664062
Epoch 8: Total loss=137.73887634277344
Epoch 9: Total loss=122.0956039428711
Epoch 10: Total loss=115.52684020996094
Epoch 11: Total loss=111.31877136230469
Epoch 12: Total loss=108.20585632324219
Epoch 13: Total loss=105.69670867919922
Epoch 14: Total loss=103.56249237060547
Epoch 15: Total loss=101.6838150024414
Epoch 16: Total loss=99.9904556274414
Epoch 17: Total loss=98.43766021728516
Epoch 18: Total loss=96.99530029296875
Epoch 19: Total loss=95.64200592041016
Epoch 20: Total loss=94.36170196533203
Epoch 21: Total loss=93.1415023803711
Epoch 22: Total loss=91.97171783447266
Epoch 23: Total loss=90.84466552734375
Epoch 24: Total loss=89.75479125976562
Epoch 25: Total loss=88.69

In [ ]:
def accuracy(model, X_test, Y_test):
    pred = model.forward(X_test).max(1)[1]
    errors = (Y_test == pred).sum().item()
    return errors / len(X_test)

In [74]:
print("Test acc:", accuracy(model, X_test, Y_test))
print("Train acc:", accuracy(model, X_train, Y_train))

Test acc: 0.981
Train acc: 0.98
